In [1]:
# Configuration
from dotenv import dotenv_values

config = dotenv_values(".env")
graphql_endpoint = config["DRUPAL_GRAPHQL_URL"]
course_acronym = config["DRUPAL_COURSE_ACRONYM"]

api_course_book = f"{config['DRUPAL_JSONAPI_URL']}/node/book"
api_course_group = f"{config['DRUPAL_JSONAPI_URL']}/group/course"
api_course_group_relationship = (
    f"{config['DRUPAL_JSONAPI_URL']}/group_relationship/course-group_node-book"
)
api_type_course_book = "node--book"
api_type_course_group = "group--course"
api_type_course_group_relationship = "group_relationship--course-group_node-book"
api_field_acronym = "field_acronym"

import os

json_filename = f"{config['DRUPAL_COURSE_ID']}.json"
course_json_path = os.path.join(config["COURSE_DATA_DIR"], json_filename)

In [2]:
# Load course json
from json import load

with open(file=course_json_path, mode="r", encoding="utf-8") as f:
    course_json = load(f)

In [3]:
from transform_course_json import apply_transformations

course_json = apply_transformations(course_json)

In [4]:
# Authentication
from requests import session

with session() as session:
    session.auth = (config["DRUPAL_USERNAME"], config["DRUPAL_PASSWORD"])
    # Headers are needed only for the JSON:API requests, but they don't impact the GraphQL ones, so it's ok to set for all
    session.headers.update(
        {
            "Accept": "application/vnd.api+json",
            "Content-Type": "application/vnd.api+json",
        }
    )

In [5]:
# List all book pages
res = session.post(
    graphql_endpoint,
    json={
        "query": """query GetAllBooks {
        books {
            id,
            title,
            body {
                value,
                summary
            },
            links {
                bid
            }
        }
    }""",
    },
)
res.json()

{'data': {'books': []}}

In [6]:
# Create new course book page
res = session.post(
    graphql_endpoint,
    json={
        "query": """mutation CreateCourseBookPage($data: BookInput!) {
        createBook(data: $data) {
            errors,
            book {
                id,
                title,
                body {
                    value,
                    summary,
                    format
                }
            }
        }
    }""",
        "variables": {
            "data": {
                "title": f"{course_acronym}: {course_json['course_name']}",
                "body": {
                    "value": course_json["overview"],
                },
            }
        },
    },
)
course_book_id = res.json()["data"]["createBook"]["book"]["id"]
res.json()

{'data': {'createBook': {'errors': [],
   'book': {'id': '31',
    'title': 'ADS: Advanced Database Systems (2022-2023)[SEM2]',
    'body': {'value': '<h3>Welcome &amp; Learning Outcomes</h3><div id="inf-welcome"> \n <h4 class="inf">Welcome to Advanced Database Systems (2022-2023)[SEM2]</h4>  \n <h5><br>Learning Outcomes</h5> \n <p>On successful completion of this course, you should be able to:&nbsp;</p> \n <p>1. Describe how database management systems function internally. Interpret and comparatively criticise database systems architectures.</p> \n <p>2. Implement major components of a database management system and analyse their performance.</p> \n <p>3. Analyse and compare the fundamental query evaluation and concurrency control algorithms, identify strengths and weaknesses of query evaluation plans, and optimise query evaluation plans.</p> \n <p>4. Identify trade-offs among database systems techniques and contrast distributed / parallel techniques for OLTP and OLAP workloads.</p> \

In [7]:
# Make the course page into a new book
res = session.post(
    graphql_endpoint,
    json={
        "query": """mutation AddCourseBookOutline($id: ID!, $data: BookInput!) {
        updateBook(id: $id, data: $data) {
            errors,
            book {
                id,
                links {
                    nid,
                    bid
                }
            }
        }
    }""",
        "variables": {
            "id": course_book_id,
            "data": {
                "title": f"{course_acronym}: {course_json['course_name']}",
                "links": {"bid": course_book_id},
            },
        },
    },
)
res.json()

{'data': {'updateBook': {'errors': [],
   'book': {'id': '31', 'links': {'nid': '31', 'bid': '31'}}}}}

In [8]:
# Get the new book page UUID
res = session.get(f"{api_course_book}?filter[drupal_internal__nid]={course_book_id}")
course_book_uuid = res.json()["data"][0]["id"]
res.json()

{'jsonapi': {'version': '1.0',
  'meta': {'links': {'self': {'href': 'http://jsonapi.org/format/1.0/'}}}},
 'data': [{'type': 'node--book',
   'id': 'f252129f-7ed7-4a78-83a3-7f6411a96e6c',
   'links': {'self': {'href': 'http://raspberrypi.local:8081/jsonapi/node/book/f252129f-7ed7-4a78-83a3-7f6411a96e6c?resourceVersion=id%3A31'}},
   'attributes': {'drupal_internal__nid': 31,
    'drupal_internal__vid': 31,
    'langcode': 'en',
    'revision_timestamp': '2023-07-19T19:40:41+00:00',
    'revision_log': None,
    'status': True,
    'title': 'ADS: Advanced Database Systems (2022-2023)[SEM2]',
    'created': '2023-07-19T19:40:41+00:00',
    'changed': '2023-07-19T19:40:41+00:00',
    'promote': False,
    'sticky': False,
    'default_langcode': True,
    'revision_translation_affected': True,
    'path': {'alias': None, 'pid': None, 'langcode': 'en'},
    'body': {'value': '<h3>Welcome &amp; Learning Outcomes</h3><div id="inf-welcome"> \n <h4 class="inf">Welcome to Advanced Database Sys

In [9]:
# Create a new group for the course
res = session.post(
    api_course_group,
    json={
        "data": {
            "type": api_type_course_group,
            "attributes": {
                "label": course_json["course_name"],
                api_field_acronym: course_acronym,
            },
        }
    },
)
course_group_uuid = res.json()["data"]["id"]
res.json()

{'jsonapi': {'version': '1.0',
  'meta': {'links': {'self': {'href': 'http://jsonapi.org/format/1.0/'}}}},
 'data': {'type': 'group--course',
  'id': '74cc7c29-7910-40d9-985f-4c6fa702ec78',
  'links': {'self': {'href': 'http://raspberrypi.local:8081/jsonapi/group/course/74cc7c29-7910-40d9-985f-4c6fa702ec78?resourceVersion=id%3A4'}},
  'attributes': {'drupal_internal__id': 4,
   'drupal_internal__revision_id': 4,
   'langcode': 'en',
   'revision_created': '2023-07-19T19:40:43+00:00',
   'revision_log_message': None,
   'status': True,
   'label': 'Advanced Database Systems (2022-2023)[SEM2]',
   'created': '2023-07-19T19:40:43+00:00',
   'changed': '2023-07-19T19:40:43+00:00',
   'path': {'alias': None, 'pid': None, 'langcode': 'en'},
   'default_langcode': True,
   'revision_translation_affected': True,
   'field_acronym': 'ADS'},
  'relationships': {'group_type': {'data': {'type': 'group_type--group_type',
     'id': 'b7bba7fe-105a-466c-9984-0303957b6883',
     'meta': {'drupal_inter

In [10]:
# Then create a relationship linking the new course book page to the course group
res = session.post(
    api_course_group_relationship,
    json={
        "data": {
            "type": api_type_course_group_relationship,
            "attributes": {
                "label": f"{course_acronym}: {course_json['course_name']}",
            },
            "relationships": {
                "entity_id": {
                    "data": {
                        "type": api_type_course_book,
                        "id": course_book_uuid,
                    }
                },
                "gid": {
                    "data": {
                        "type": api_type_course_group,
                        "id": course_group_uuid,
                    }
                },
            },
        }
    },
)
# If you get HTTP 500 error, see note in the README file
res.json()

{'jsonapi': {'version': '1.0',
  'meta': {'links': {'self': {'href': 'http://jsonapi.org/format/1.0/'}}}},
 'data': {'type': 'group_relationship--course-group_node-book',
  'id': '40e58f47-dd62-4801-94fc-9f18950d2674',
  'links': {'self': {'href': 'http://raspberrypi.local:8081/jsonapi/group_relationship/course-group_node-book/40e58f47-dd62-4801-94fc-9f18950d2674'}},
  'attributes': {'drupal_internal__id': 36,
   'langcode': 'en',
   'label': 'ADS: Advanced Database Systems (2022-2023)[SEM2]',
   'created': '2023-07-19T19:40:46+00:00',
   'changed': '2023-07-19T19:40:46+00:00',
   'plugin_id': 'group_node:book',
   'path': {'alias': None, 'pid': None, 'langcode': 'en'},
   'default_langcode': True},
  'relationships': {'group_relationship_type': {'data': None,
    'links': {'self': {'href': 'http://raspberrypi.local:8081/jsonapi/group_relationship/course-group_node-book/40e58f47-dd62-4801-94fc-9f18950d2674/relationships/group_relationship_type'}}},
   'uid': {'data': {'type': 'user--us

In [11]:
# Now add pages for all the course content
def link_book_page_to_group(page_id, title, group_uuid):
    res = session.get(f"{api_course_book}?filter[drupal_internal__nid]={page_id}")
    page_uuid = res.json()["data"][0]["id"]
    res = session.post(
        api_course_group_relationship,
        json={
            "data": {
                "type": api_type_course_group_relationship,
                "attributes": {
                    "label": f"{course_acronym}: {title}",
                },
                "relationships": {
                    "entity_id": {
                        "data": {
                            "type": api_type_course_book,
                            "id": page_uuid,
                        }
                    },
                    "gid": {
                        "data": {
                            "type": api_type_course_group,
                            "id": group_uuid,
                        }
                    },
                },
            }
        },
    )
    print(res.json())


def create_outline_page(title, book_id, weight):
    res = session.post(
        graphql_endpoint,
        json={
            "query": """mutation CreateCourseOutlinePage($data: BookInput!) {
            createBook(data: $data) {
                errors,
                book {
                    id,
                    title,
                    links {
                        bid,
                        pid,
                        weight,
                    }
                }
            }
        }""",
            "variables": {
                "data": {
                    "title": f"{course_acronym}: {title}",
                    "links": {
                        "bid": book_id,
                        "pid": book_id,
                        "weight": weight,
                    },
                }
            },
        },
    )
    print(res.json())
    return res.json()["data"]["createBook"]["book"]["id"]


def create_content_page(title, summary, body, book_id, parent_id, weight):
    res = session.post(
        graphql_endpoint,
        json={
            "query": """mutation CreateCourseContentPage($data: BookInput!) {
            createBook(data: $data) {
                errors,
                book {
                    id,
                    title,
                    body {
                        value,
                        summary,
                        format,
                    },
                    links {
                        bid,
                        pid,
                        weight,
                    }
                }
            }
        }""",
            "variables": {
                "data": {
                    "title": f"{course_acronym}: {title}",
                    "body": {
                        "summary": summary,
                        "value": body,
                    },
                    "links": {
                        "bid": book_id,
                        "pid": parent_id,
                        "weight": weight,
                    },
                }
            },
        },
    )
    print(res.json())
    return res.json()["data"]["createBook"]["book"]["id"]


def create_content_pages_rec(toc_subitem, book_id, parent_id, weight, group_uuid):
    page_id = create_content_page(
        toc_subitem["title"],
        toc_subitem["description"],
        toc_subitem["body"],
        book_id,
        parent_id,
        weight,
    )
    link_book_page_to_group(page_id, toc_subitem["title"], group_uuid)
    for child_index, child in enumerate(toc_subitem["children"]):
        create_content_pages_rec(child, book_id, page_id, child_index, group_uuid)


# Add a content pages linked to the course book
for toc_index, toc_item in enumerate(course_json["toc"]):
    toc_page_id = create_outline_page(toc_item["title"], course_book_id, toc_index)
    link_book_page_to_group(toc_page_id, toc_item["title"], course_group_uuid)
    for child_index, child in enumerate(toc_item["children"]):
        create_content_pages_rec(
            child, course_book_id, toc_page_id, child_index, course_group_uuid
        )

{'data': {'createBook': {'errors': [], 'book': {'id': '32', 'title': 'ADS: Course Materials', 'links': {'bid': '31', 'pid': '31', 'weight': 0}}}}}
{'jsonapi': {'version': '1.0', 'meta': {'links': {'self': {'href': 'http://jsonapi.org/format/1.0/'}}}}, 'data': {'type': 'group_relationship--course-group_node-book', 'id': 'e2bc9384-1357-44d2-af69-4668b4fa587c', 'links': {'self': {'href': 'http://raspberrypi.local:8081/jsonapi/group_relationship/course-group_node-book/e2bc9384-1357-44d2-af69-4668b4fa587c'}}, 'attributes': {'drupal_internal__id': 37, 'langcode': 'en', 'label': 'ADS: Course Materials', 'created': '2023-07-19T19:40:48+00:00', 'changed': '2023-07-19T19:40:48+00:00', 'plugin_id': 'group_node:book', 'path': {'alias': None, 'pid': None, 'langcode': 'en'}, 'default_langcode': True}, 'relationships': {'group_relationship_type': {'data': None, 'links': {'self': {'href': 'http://raspberrypi.local:8081/jsonapi/group_relationship/course-group_node-book/e2bc9384-1357-44d2-af69-4668b4fa5